<a href="https://colab.research.google.com/github/Cyanjiner/classroom-analysis/blob/main/transcript_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab workspace
from google.colab import drive
drive.mount('/content/drive')

# installations for VMs
# !pip install transformers
# # !pip install -U sentence-transformers
# !pip install sentencepiece
!pip install textacy
!pip install pdfminer

import math, sys, os, requests, datetime, json, re, csv, io, copy
import numpy as np
import pandas as pd
## Text structures
import textblob
import nltk
import spacy
spacy.prefer_gpu()
# spacy.require_gpu()
# !python -m spacy download en_core_web_trf
import textacy
import textacy.text_stats
import textacy.preprocessing
from typing import Literal
from bs4 import BeautifulSoup
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams

import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import smart_open
import gensim
# from node2vec import Node2Vec
import networkx as nx
from nltk.tokenize import word_tokenize

import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from string import ascii_lowercase

ROOT_FOLDER = "/content/drive/My Drive/transcript_analysis/"
DATA_FOLDER = ROOT_FOLDER + "data/"
RAW_FOLDER = DATA_FOLDER + "raw/"
CLEAN_FOLDER = DATA_FOLDER + "clean/"
MODELS_FOLDER = ROOT_FOLDER + "models/"
LOG_DIR = MODELS_FOLDER + "runs/"

In [ ]:
df = pd.read_csv(DATA_FOLDER + "ncte_single_utterances.csv")

In [ ]:
## Create standards from for grades 1 to 6 into a data frame
sdf = pd.DataFrame(columns = ['Subject', 'Grade', 'Domain', 'Cluster Statement', 'Standard Code',
       'Common Core Standard'])
for grade in range(1,9):
  sdf = pd.concat([sdf,pd.read_excel(DATA_FOLDER + "math-ccss-k-12.xlsx", sheet_name=f"Grade {grade}")])

sdf = pd.concat([sdf,pd.read_excel(DATA_FOLDER + "math-ccss-k-12.xlsx", sheet_name=f"Kinder")])
sdf.columns = [x.replace(" ","_").lower() for x in sdf.columns]

sdf

,subject,grade,domain,cluster_statement,standard_code,common_core_standard,common_core_state_standard
0,Mathematics,Grade 1,Mathematical Practices,The Standards for Mathematical Practice descri...,1.MP.1,Make sense of problems and persevere in solvin...,NaN
1,Mathematics,Grade 1,Mathematical Practices,The Standards for Mathematical Practice descri...,1.MP.2,Reason abstractly and quantitatively.,NaN
2,Mathematics,Grade 1,Mathematical Practices,The Standards for Mathematical Practice descri...,1.MP.3,Construct viable arguments and critique the re...,NaN
3,Mathematics,Grade 1,Mathematical Practices,The Standards for Mathematical Practice descri...,1.MP.4,Model with mathematics.,NaN
4,Mathematics,Grade 1,Mathematical Practices,The Standards for Mathematical Practice descri...,1.MP.5,Use appropriate tools strategically.,NaN
...,...,...,...,...,...,...,...
28,Mathematics,Kindergarten,Geometry,"Identify and describe shapes (squares, circles...",K.G.2,Correctly name shapes regardless of their orie...,NaN
29,Mathematics,Kindergarten,Geometry,"Identify and describe shapes (squares, circles...",K.G.3,Identify shapes as two-dimensional (lying in a...,NaN
30,Mathematics,Kindergarten,Geometry,"Analyze, compare, create, and compose shapes.",K.G.4,Analyze and compare two- and three-dimensional...,NaN
31,Mathematics,Kindergarten,Geometry,"Analyze, compare, create, and compose shapes.",K.G.5,Model shapes in the world by building shapes f...,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580408 entries, 0 to 580407
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   speaker       580395 non-null  object
 1   text          580408 non-null  object
 2   year          580408 non-null  int64 
 3   OBSID         580408 non-null  int64 
 4   video_id      580408 non-null  int64 
 5   cleaned_text  580358 non-null  object
 6   num_words     580408 non-null  int64 
 7   turn_idx      580408 non-null  int64 
 8   comb_idx      580408 non-null  object
dtypes: int64(5), object(4)
memory usage: 39.9+ MB


In [ ]:
# pd.crosstab(df.)

,speaker,text,year,OBSID,video_id,cleaned_text,num_words,turn_idx,comb_idx
0,teacher,"Friends, yesterday we started off by working o...",2,2119,706,friends yesterday we started off by working on...,13,0,2119_0
1,student,Yes.,2,2119,706,yes,1,1,2119_1
2,teacher,"And yesterday towards the end of the period, y...",2,2119,706,and yesterday towards the end of the period yo...,38,2,2119_2
3,multiple students,Yes.,2,2119,706,yes,1,3,2119_3
4,teacher,Some of you might be done. If you are finishe...,2,2119,706,some of you might be done if you are finished ...,106,4,2119_4


In [ ]:
df.groupby(by=["OBSID"]).sum()

,year,video_id,num_words,turn_idx
OBSID,,,,
3,250,82250,4925,31125
4,465,152055,8156,107880
5,591,192666,6849,174345
7,523,178343,5859,136503
8,468,163800,6674,109278
...,...,...,...,...
4758,381,54356,3013,8001
4759,1233,220707,3055,84255
4760,2997,440559,8266,498501


In [ ]:
df.groupby(by=["OBSID","video_id"]).size()

OBSID  video_id
3      329          250
4      327          465
5      326          591
7      341          523
8      350          468
                   ... 
4758   428          127
4759   537          411
4760   441          999
4762   444          390
4763   832         1158
Length: 1661, dtype: int64

In [ ]:
df.video_id.value_counts()

729     4554
784     4485
419     4456
760     4376
443     4240
        ... 
513      284
539      282
548      279
1411     279
557      109
Name: video_id, Length: 319, dtype: int64

In [ ]:
df.speaker.value_counts()

teacher              286561
student              255268
multiple students     38566
Name: speaker, dtype: int64

In [ ]:
asdf = df[df.video_id == 706]
asdf

,speaker,text,year,OBSID,video_id,cleaned_text,num_words,turn_idx,comb_idx
0,teacher,"Friends, yesterday we started off by working o...",2,2119,706,friends yesterday we started off by working on...,13,0,2119_0
1,student,Yes.,2,2119,706,yes,1,1,2119_1
2,teacher,"And yesterday towards the end of the period, y...",2,2119,706,and yesterday towards the end of the period yo...,38,2,2119_2
3,multiple students,Yes.,2,2119,706,yes,1,3,2119_3
4,teacher,Some of you might be done. If you are finishe...,2,2119,706,some of you might be done if you are finished ...,106,4,2119_4
...,...,...,...,...,...,...,...,...,...
389930,teacher,How was pi created? We're actually going to d...,1,266,706,how was pi created we re actually going to dis...,16,170,266_170
389931,student,What if we run out of yarn?,1,266,706,what if we run out of yarn,7,171,266_171
389932,teacher,"I'll give you some more in a minute. Yes, Stu...",1,266,706,i ll give you some more in a minute yes student k,12,172,266_172
389933,student,Who made it up?,1,266,706,who made it up,4,173,266_173


In [ ]:
asdf.OBSID.value_counts()

626     375
2119    323
2639    306
2044    304
266     175
4039    145
760      94
4130     87
4717     84
Name: OBSID, dtype: int64

In [ ]:
ddf = df[df.OBSID == 2119]
ddf

,speaker,text,year,OBSID,video_id,cleaned_text,num_words,turn_idx,comb_idx
0,teacher,"Friends, yesterday we started off by working o...",2,2119,706,friends yesterday we started off by working on...,13,0,2119_0
1,student,Yes.,2,2119,706,yes,1,1,2119_1
2,teacher,"And yesterday towards the end of the period, y...",2,2119,706,and yesterday towards the end of the period yo...,38,2,2119_2
3,multiple students,Yes.,2,2119,706,yes,1,3,2119_3
4,teacher,Some of you might be done. If you are finishe...,2,2119,706,some of you might be done if you are finished ...,106,4,2119_4
...,...,...,...,...,...,...,...,...,...
318,teacher,Are what?,2,2119,706,are what,2,318,2119_318
319,student,Are not.,2,2119,706,are not,2,319,2119_319
320,teacher,Are not wearing ties. How many – so you don't...,2,2119,706,are not wearing ties how many so you don t wan...,171,320,2119_320
321,student,Where's the pictures?,2,2119,706,where s the pictures,4,321,2119_321


# Load all the corpora

In [ ]:
ROOT_FOLDER = "/content/drive/My Drive/transcript_analysis/"
DATA_FOLDER = ROOT_FOLDER + "data/"
RAW_FOLDER = DATA_FOLDER + "raw/"
CLEAN_FOLDER = DATA_FOLDER + "clean/"

ssdf = pd.read_csv(DATA_FOLDER + "ccss_text.csv")
imdf = pd.read_csv(DATA_FOLDER + "IM_text.csv")
tdf = pd.read_csv(DATA_FOLDER + "talk_math.csv")
gdf = pd.read_csv(DATA_FOLDER + "math_glossary.csv")
sfdf = pd.read_csv(DATA_FOLDER + "SFUSD/sfusd_mathtalks.csv")

Index(['speaker', 'text', 'year', 'OBSID', 'video_id', 'cleaned_text',
       'num_words', 'turn_idx', 'comb_idx'],
      dtype='object')

# Accessing Academic Language by using the links found in the spreadsheet

In [ ]:
# test = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
#                    '0Ak1ecr7i0wotdGJmTURJRnZLYlV3M2daNTRubTdwTXc' +
#                    '/export?gid=0&format=csv',
#                    # Set first column as rownames in data frame
#                    index_col=0,
#                    # Parse column values to datetime
#                    parse_dates=['Quradate']
#                   )
# test.head(5)  # Same resul

In [ ]:
## Getting exemplary language
url = f"https://docs.google.com/spreadsheets/d/13e28Y7l4sAQ5p-TEkRIb5MJ-T58Mba5ZMp7PaBBjWKQ/edit#gid=1086911083"
url_1 = url.replace('/edit#gid=', '/export?format=csv&gid=')
# url = f"https://docs.google.com/spreadsheets/d/13e28Y7l4sAQ5p-TEkRIb5MJ-T58Mba5ZMp7PaBBjWKQ/edit#gid=1086911083"
ldf = pd.read_csv(url_1)

In [ ]:
ldf

,UNIT CODE,UNIT TITLE,LESSON CODE,LESSON TITLE,ACTIVITY ADDRESS,ACTIVITY TITLE,IM Demo Site url,KH url,Act Number Formula,Course,Activity Title,Unit Number,Lesson Number,Activity Number,Instructional Routines,IM Demo hyperlink,KH hyperlink
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Math in Our World,1.0,Explore Connecting Cubes,K5_v1.1.1.1,Notice and Wonder: Connecting Cubes,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/kinderg...,WU,Kindergarten,Notice and Wonder: Connecting Cubes,1.0,1.0,1.WU,Notice and Wonder,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/kinderg...
2,1.0,Math in Our World,1.0,Explore Connecting Cubes,K5_v1.1.1.2,"Introduce Connecting Cubes, Explore",https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/kinderg...,1,Kindergarten,"Introduce Connecting Cubes, Explore",1.0,1.0,1.1,NaN,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/kinderg...
3,1.0,Math in Our World,2.0,Explore Pattern Blocks,K5_v1.1.2.1,Notice and Wonder: Pattern Blocks,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/kinderg...,WU,Kindergarten,Notice and Wonder: Pattern Blocks,1.0,2.0,2.WU,Notice and Wonder,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/kinderg...
4,1.0,Math in Our World,2.0,Explore Pattern Blocks,K5_v1.1.2.2,"Introduce Pattern Blocks, Explore",https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/kinderg...,1,Kindergarten,"Introduce Pattern Blocks, Explore",1.0,2.0,2.1,NaN,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/kinderg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2839,50.0,Putting It All Together,17.0,True or False?,K5_v1.50.17.4,Design 3,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/grade-5...,3,Grade 5,Design 3,8.0,17.0,17.3,NaN,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/grade-5...
2840,50.0,Putting It All Together,18.0,Which One Doesn’t Belong?,K5_v1.50.18.1,Which One Doesn’t Belong? Volume,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/grade-5...,WU,Grade 5,Which One Doesn’t Belong? Volume,8.0,18.0,18.WU,Which One Doesn’t Belong?,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/grade-5...
2841,50.0,Putting It All Together,18.0,Which One Doesn’t Belong?,K5_v1.50.18.2,Which One Doesn't Belong? Design 1,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/grade-5...,1,Grade 5,Which One Doesn't Belong? Design 1,8.0,18.0,18.1,NaN,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/grade-5...
2842,50.0,Putting It All Together,18.0,Which One Doesn’t Belong?,K5_v1.50.18.3,Which One Doesn't Belong? Design 2,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/grade-5...,2,Grade 5,Which One Doesn't Belong? Design 2,8.0,18.0,18.2,NaN,https://curriculum.illustrativemathematics.org...,https://im.kendallhunt.com/k5/teachers/grade-5...


In [ ]:
##
tdf = {}
grades = list(range(1,6))
pat = re.compile("“.+”\s{0,3}\(.+\)")
df_cols = ['grade','lesson','title','standard','ts_text','t_guide_all','url','material','full_page_text']
imdf = pd.DataFrame(columns = df_cols)

gr = 5
for gr in grades:
  for i, row in ldf[ldf.Course == f"Grade {gr}"].iterrows():
    url = row["IM Demo hyperlink"]
    # url = ldf[ldf.Course == f"Grade {gr}"].iloc[0]["IM Demo hyperlink"]
    r1 = requests.get(url)
    page = r1.content
    page_data = BeautifulSoup(page, 'html5lib')
    t = page_data.find('main')

    tdf['standard'] = [a.text for a in t.find_all('a')][2]
    tdf['lesson'] = [a.text for a in t.find_all('h1')][0]
    tdf['title'] = row["Activity Title"]
    tdf['module'] = row['Unit Number']
    tdf['grade'] = gr
    tdf['ts_text'] = [pat.findall(t.text)]
    tdf['t_guide_all'] = [[a.text for a in t.find_all('li')]]
    tdf['url'] = url
    tdf['material'] = "lesson_activity"
    tdf['full_text'] = t.text
    imdf = pd.concat([imdf,pd.DataFrame(tdf)])

# print(t.prettify())
imdf.to_csv(DATA_FOLDER + "IM_text.csv")
imdf

In [ ]:
[a.text for a in t.find_all('li')]

['5.MD.C.3',
 'Which One Doesn’t Belong?',
 'After the warm-up, ask students to reflect on both individual and group actions while considering the question “What does it look and sound like to do math together as a mathematical community? What am I doing? What are you doing?”',
 'Record and display their responses under the “Doing Math” header. Students might mention things such as: we talked to each other and to the teacher; we had quiet time to think; we shared our ideas; we thought about the math ideas and words we knew; you were writing down our answers; you were waiting until we gave the answers.',
 'Groups of \xa02',
 'Display image',
 '“Pick one that doesn’t belong. Be ready to share why it doesn’t belong.”',
 '1 minute: quiet think time',
 '2-3 minutes: partner discussion',
 'Share and record responses.\xa0',
 '“What do objects A, C, and D have in common?” (Each side or face of the shapes is a rectangle. They are all made from cubes of the same size.)',
 '5.MD.C.3',
 'MLR2',
 '

In [ ]:
# import re
# print(page_data.prettify())
# for d in t.find_all('div'):
#   print(d.text)

pat = re.compile("“.+”\s{0,3}\(.+\)")
results = pat.findall(t.text)
results

['“What do objects A, C, and D have in common?” (Each side or face of the shapes is a rectangle. They are all made from cubes of the same size.)',
 '“How would you describe the amount of space each object takes up?” (It’s the same for both shapes. They are made from 8 individual cubes.)',
 '“Which object\xa0has greater volume?” (They are the same. They are both made from 8 cubes.)',
 '“What is different about the two objects?” (One is a cube and the other is not. One is taller than the other.)',
 '“What is the same? What is different?” (Some of the objects are taller, some are shorter. Some of the objects are wider. They are all made with 9 cubes.)',
 '“How do the volumes of these objects compare? How do you know?” (They are all the same because each object\xa0takes up the same amount of space or has the same number of cubes.)',
 '“How much space does each object\xa0take up?” (9 cubes)',
 '“How are the two objects the same?” (They are both made from 8 cubes. There are four cubes next t

In [ ]:
tt = t.text
pat = re.compile("“.+”\s\(.+\)")
results = pat.findall(tt)
results

NameError: ignored

In [ ]:
## Build a dataframe with the pertinent info by row

asdf = {}

asdf['standard'] = [a.text for a in t.find_all('a')][2]
asdf['grade'] = gr




In [ ]:
[a.text for a in t.find_all('a')][2]

'5.MD.C.3'

In [ ]:
" sdfaljkhsdf"(alsfkdjhlakjsdhf)

In [ ]:

tb = page_data.find_all('div')
tb = page_data.find_all(id="im-c-content")
len(tb)

0

## SFUSD

In [ ]:
!pip install python-docx 
import docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=6aa3ecec326a2f584e46d62fee1bc86810d60671a5b10477de20b444a783a646
  Stored in directory: /root/.cache/pip/wheels/32/b8/b2/c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


In [ ]:
doc = docx.Document(DATA_FOLDER + "SFUSD/" + "Grade 3-5 Comprehensive Math Talks Bank.docx" )

In [ ]:
cols_df = ['grade','lesson','objective','description','teacher_talk','student_talk']
sfdf = pd.DataFrame(columns = cols_df)

sr_flag = False
ds_flag = False
obj_flag = False
t_flag = False
un_pat = re.compile('unit \d')
gr_pat = re.compile('grade \d')
obj_pat = re.compile('objective.*')
st_res_pat = re.compile('student responses{0,1}.*')
prev_grade = "Grade 4"
prev_objective = ''
prev_lesson = ''
for i, t in enumerate(doc.tables):
  for j, col in enumerate(t.columns):
    new_entry = {}
    description = []
    t_talk = []
    s_talk = []
    for k, cell in enumerate(col.cells):
      cell_contents = cell.text.lower()
      grade = un_pat.findall(cell_contents) if un_pat.findall(cell_contents) else gr_pat.findall(cell_contents)
      if not grade: grade = prev_grade
      new_entry['grade'] = grade[-1]
      new_entry['objective'] = obj_pat.findall(cell_contents)[0] if obj_pat.findall(cell_contents) else prev_objective
      # print(objective)
      # if objective: new_entry['objective'] = objective
      # else: new_entry['objective'] = prev_objective

      pp = [x for x in cell.text.split("\n") if x not in ('',' ',)]
      for l, text in enumerate(pp):
        if "extension" in text.lower():
          lesson = text.lower().split(":")[-1]
        elif '?' in text or 'ask' in text.lower():
          t_talk.append(text)
          sr_flag = True
        elif "source" in text.lower() or "resources" in text.lower() : 
          break
        elif 'objective:' in text.lower() or 'objective:' in pp[max(l-1,0)].lower(): 
          lesson = pp[l-1] if l-1>=0 else prev_lesson
          
        elif 'description:' in text.lower():
          description.append(text)
          ds_flag = True
          sr_flag = False
        elif st_res_pat.findall(text) or sr_flag:
          s_talk.append(text)
          sr_flag = True
          ds_flag = False
        else: description.append(text)
      prev_objective = new_entry['objective']

    ds_flag = False
    sr_flag = False
    prev_grade = grade
    
    new_entry['lesson'] = description[1] if len(description) > 1 else prev_lesson
    new_entry['description'] = "\n".join(description)
    new_entry['teacher_talk'] = "\n".join(t_talk)
    new_entry['student_talk'] = "\n".join(s_talk)
    if not lesson: new_entry['lesson'] = prev_lesson
    prev_lesson = lesson


    sfdf = pd.concat([sfdf,pd.DataFrame(new_entry,index = [f"t{i}"])]).drop_duplicates(subset =cols_df[:-1])

    

sfdf.to_csv(DATA_FOLDER + "SFUSD/sfusd_mathtalks.csv")
sfdf

,grade,lesson,objective,description,teacher_talk,student_talk
t0,4,Dot Talks,objective: students explain their strategies f...,Re-engage with Previous Content\nDot Talks\nAs...,Question/Prompt: How many dots do you see? How...,Anticipated Student Responses:\nI saw 4 dots i...
t1,unit 4,Money Comparisons,objective: students use their knowledge of the...,Re-engage with Previous Content\nMoney Compari...,Description: Ask students Which amount is grea...,Anticipated Student Responses: \n62 pennies is...
t2,unit 4,Expressions Talks,objective: students organize visual patterns a...,Re-engage with Previous Content\nExpressions T...,How many ___ do you see? How do you see them? ...,Anticipated Student Response:\nI see 3 rows. A...
t2,unit 4,Expressions Talks,objective: students organize visual patterns a...,Re-engage with Previous Content\nExpressions T...,How many ___ do you see? How do you see them? ...,
t3,unit 4,Which One Doesn’t Belong,objective: students use their knowledge of pla...,Engage with Current Content\nWhich One Doesn’t...,Question/Prompt: Which one doesn’t belong? Why?,Anticipated Student Responses: \nStudents may ...
...,...,...,...,...,...,...
t41,unit 4,Number Line Talks,objective: analyze shape and number patterns t...,5th Grade Math Talks\nNumber Line Talks,What expression might go with this number line...,Students might say:\n ...
t41,unit 4,Fractions Strings,objective: analyze shape and number patterns t...,5th Grade Math Talks\nFractions Strings\nAddit...,,
t42,unit 4,Objective: Analyze shape and number patterns t...,objective: analyze shape and number patterns t...,Fractions in Venn Diagrams ...,Where does each number go in this diagram?\nAb...,Division of Decimals by Decimals - Estimation ...
t42,unit 4,Objective: Analyze shape and number patterns t...,objective: analyze shape and number patterns t...,Fractions in Venn Diagrams ...,Where does each number go in this diagram?\nWi...,Division of Decimals by Decimals - Estimation ...


In [ ]:
"colwise

[['Re-engage with Previous Content',
  'Expressions Talks\nObjective: Students organize visual patterns and describe them as mathematical expressions.\nDescription: Show the image and give students a few minutes to think about how to visually organize the shapes, then share their organization with an expressions. As students share, record their organizations on copies of the math talk and record their expressions.\n\nHow many ___ do you see? How do you see them? What expression (or equation) represents this?',
  '',
  '',
  '',
  'Anticipated Student Response:\n\nI see 3 rows. A row of 4, a row of 3, and another row of 4. \n4 + 3 + 4 = 11.\nI see 2 rows of 4 and 1 row of 3.\n(2 x 4) + 3  = 11.\n\nI see 3 groups of 3 diagonals. 3 + 3 + 3 = 9, or 3 x 3 = 9\nThere are 2 extra dots. 9 + 2 = 11\n\nOr (3 + 3 + 3) + 1 = 11; or (3 x 3) + 1 = 11',
  'Resources: '],
 ['Re-engage with Previous Content',
  'Expressions Talks\nObjective: Students organize visual patterns and describe them as mathem

In [ ]:
rowwise

[['Re-engage with Previous Content',
  'Re-engage with Previous Content',
  'Re-engage with Previous Content'],
 ['Dot Talks\nObjective: Students explain their strategies for grouping dots in various formations.\nDescription: Kindergarten students begin learning how to subitize, or ‘see’ a small number of objects, and know how many there are without counting. When looking at a group of dots, they may see groups of dots and combine them to find the total number without counting one by one. This helps students learn strategies for composition and decomposition, and the properties of operations. As students learn the operations, they can relate how they see groups of dots to expressions and equations that represent this. \nAs students describe their thinking, record their description both on the dot formations and using more formal expressions so students begin to see the relationship between the two.',
  'Dot Talks\nObjective: Students explain their strategies for grouping dots in variou

# Extract the "Talking Math" text information

In [ ]:
with open(DATA_FOLDER + "talking_math.txt") as f:
    lines = f.readlines()
lines = [x.strip() for x in lines if x.strip() != '']
lines

['Talking Math',
 'Launch',
 'What math questions could you ask about this picture?',
 'Grade K',
 'What kinds of toys do you see?',
 'What can you tell me about the toys?',
 'Invitational 1',
 'Link to the picture',
 'Grade K',
 'What kinds of toys do you see?',
 'What can you tell me about the toys?',
 'Grade 1',
 'How many toys are round? How many have straight sides?',
 'Do you think there are more round toys or toys with straight sides? Why?',
 '(K.MD and 1.MD)',
 'Grade 2',
 'Complete the sentences:',
 'There are 2 more ________ than _______.',
 'There are 8 __________.',
 'In this picture there are groups of___________.',
 'The number of ___________ is less than the number of __________.',
 '(1.MD and 2.MD.10)',
 'Grade 3',
 'Pick four kinds of toys and count them.',
 'If you made a picture graph about the number of each kind, what could you use as a symbol for each toy?',
 '(2.MD)',
 'Talking Math',
 'Invitational 1',
 'Focus on Data',
 'Note: Invitational prompts are sometimes

In [ ]:
grades = ['Grade ' + str(x) for x in ['K',1,2,3,4,5]]
dfcols = ['grade','standard','text','text_type','lesson']
standards = list(ssdf.index)

tdf = pd.DataFrame(columns = dfcols)
text_type = "t_questions"
prev_text, text = [""],[""]
prev_grade,grade,prev_std,std,lesson = ['']*5
lesson = "Invitational"
pat = re.compile("\(.+\)") ## find all the standards
is_grade = False
for line in lines:
  if line[:3] == 'Day':    
    lesson = line.split(" ")[-1]
    is_grade = False
    text = [line]
  if prev_text[0] in grades or is_grade:
    if is_grade: text.append(line)
    else: text = [line]
    is_grade = True
    if prev_text[0] in grades: grade = prev_text[0]
    parens = pat.findall(line)
    if parens: 
      parens = [[s for s in x.strip()[1:-1].split(' ') if s in standards] for x in parens]
      tdf = pd.concat([tdf,pd.DataFrame({'grade':grade,'standard':parens,
                                         'text':[text],'text_type':text_type,
                                         "lesson":lesson})])
      is_grade = False
      grade = ""
      text = [line]
  elif is_grade:
    text.append(line)
  else:
    text = [line]
  prev_text = copy.copy(text)
tdf.to_csv(DATA_FOLDER + "talk_math.csv")
tdf

,grade,standard,text,text_type,lesson
0,Grade K,[],"[What kinds of toys do you see?, What can you ...",t_questions,Invitational
0,Grade 2,[],"[Complete the sentences:, There are 2 more ___...",t_questions,Invitational
0,Grade 3,[],"[Pick four kinds of toys and count them., If y...",t_questions,Invitational
0,Grade 4,[],[These are the lengths of the cars in the pict...,t_questions,Invitational
0,Grade 5,[],[These are the lengths of the blocks in the pi...,t_questions,Invitational
...,...,...,...,...,...
0,Grade 1,[1.MD.A.2],[Find some of these objects in your home. How ...,t_questions,95
0,Grade 2,[2.MD.A.1],[There are rulers in this picture. What can yo...,t_questions,95
0,Grade 3,[3.MD.B.4],[Find 3 of the objects similar to those in the...,t_questions,95
0,Grade 4,[4.MD.A.1],[The longer paint brushes in the picture are a...,t_questions,95


In [ ]:
list(ssdf.index)

['K.CC.A.1',
 'K.CC.A.2',
 'K.CC.A.3',
 'K.CC.B.4',
 'K.CC.B.5',
 'K.CC.C.6',
 'K.CC.C.7',
 'K.CC.A',
 'K.CC.B',
 'K.CC.C',
 'K.NBT.A.1',
 '1.NBT.A.1',
 '1.NBT.B.2',
 '1.NBT.B.3',
 '1.NBT.C.4',
 '1.NBT.C.5',
 '1.NBT.C.6',
 '2.NBT.A.1',
 '2.NBT.A.2',
 '2.NBT.A.3',
 '2.NBT.A.4',
 '2.NBT.B.5',
 '2.NBT.B.6',
 '2.NBT.B.7',
 '2.NBT.B.8',
 '2.NBT.B.9',
 '3.NBT.A.1',
 '3.NBT.A.2',
 '3.NBT.A.3',
 '4.NBT.A.1',
 '4.NBT.A.2',
 '4.NBT.A.3',
 '4.NBT.B.4',
 '4.NBT.B.5',
 '4.NBT.B.6',
 '5.NBT.A.1',
 '5.NBT.A.2',
 '5.NBT.A.3',
 '5.NBT.A.4',
 '5.NBT.B.5',
 '5.NBT.B.6',
 '5.NBT.B.7',
 'K.NBT.A',
 '1.NBT.A',
 '1.NBT.B',
 '1.NBT.C',
 '2.NBT.A',
 '2.NBT.B',
 '3.NBT.A',
 '4.NBT.A',
 '4.NBT.B',
 '5.NBT.A',
 '5.NBT.B',
 '6.RP.A.1',
 '6.RP.A.2',
 '6.RP.A.3',
 '7.RP.A.1',
 '7.RP.A.2',
 '7.RP.A.3',
 '6.RP.A',
 '7.RP.A',
 '6.NS.A.1',
 '6.NS.B.2',
 '6.NS.B.3',
 '6.NS.B.4',
 '6.NS.C.5',
 '6.NS.C.6',
 '6.NS.C.7',
 '6.NS.C.8',
 '7.NS.A.1',
 '7.NS.A.2',
 '7.NS.A.3',
 '8.NS.A.1',
 '8.NS.A.2',
 '6.NS.A',
 '6.NS.B',
 '6.N

In [ ]:
url = "https://web.archive.org/web/20220206180509/http://www.corestandards.org/Math/Content/mathematics-glossary/"
r1 = requests.get(url)
page = r1.content
glossary_page = BeautifulSoup(page, 'html5lib')
tt = glossary_page.find('section',"content clearfix")
math_glossary = {}
for dfn in [x for x in glossary_page.find_all('p')]:
  term = dfn.find('strong')
  if term: 
    termt = term.text.strip()
    if termt[-1] == '.':
      termt = termt[:-1]
    math_glossary[termt] = dfn.text[len(term.text):].strip()
g = pd.DataFrame(math_glossary.values(),columns = ['definition'], index = math_glossary.keys())
g.to_csv(DATA_FOLDER + "math_glossary.csv")

# This super complicated set of code is to get the CCSS standards
There were discrepancies with various non-official source, and the official website wasn't functioning, and using archive.org was a pain.

In [ ]:
ssdf = pd.DataFrame(columns = ["grade",
                        "domain",
                        "area",
                        "strand",
                        "substrand",
                        "cluster",
                        "text",
                        "examples",
                        "full_text"])

urls = ["https://web.archive.org/web/20220207043616/http://www.corestandards.org/Math/Content/G/",
        "https://web.archive.org/web/20220207043606/http://www.corestandards.org/Math/Content/MD/",
        "https://web.archive.org/web/20220206180615/http://www.corestandards.org/Math/Content/CC/",
        "https://web.archive.org/web/20220206180636/http://www.corestandards.org/Math/Content/NBT/",
        "https://web.archive.org/web/20220206180646/http://www.corestandards.org/Math/Content/NF/",
        "https://web.archive.org/web/20220207043626/http://www.corestandards.org/Math/Content/RP/",
        "https://web.archive.org/web/20220207043646/http://www.corestandards.org/Math/Content/NS/",
        "https://web.archive.org/web/20220207043658/http://www.corestandards.org/Math/Content/EE/",
        "https://web.archive.org/web/20220207043708/http://www.corestandards.org/Math/Content/F/",
        "https://web.archive.org/web/20220207043719/http://www.corestandards.org/Math/Content/SP/",
        "https://web.archive.org/web/20220206180624/http://www.corestandards.org/Math/Content/OA/"

        ]

try_again = []
do_scraping = True
while do_scraping:
  for url in urls:
    try:
      r1 = requests.get(url)
    except:
      try_again.append(url)  
      print(url[-4:])
      continue
    page_data = BeautifulSoup(r1.content, 'html5lib')
    tt = page_data.find('section')
    stds = {}
    for s in [x for x in tt.find_all('div',"standard")]:
      examples, subs = ['']*2
      standard = s.find('a').text.strip()
      examples = " ".join([i.text for i in s.find_all('i') if s.find('i')]).strip()
      subs = s.find('sup')
      # examples = examples.text.strip() if examples else ''
      subs = subs.text.strip() if subs else ''
      assert len(s.text.strip()) > (len(standard) + (len(examples)+len(subs)))
      language = s.text[len(standard):]
      standard = standard[18:]
      breakdown = standard.split(".")
      if len(breakdown) == 5: grade, section, subsection, standard_num, substandard = breakdown
      else: 
        grade, section, subsection, standard_num = breakdown
        substandard = ""
      stds[standard] = {"grade":grade,
                        "domain":section,
                        "area":subsection,
                        "strand":standard_num,
                        "substrand":substandard,
                        "text":language,
                        "examples":examples,
                        "full_text":s.text,
                        "cluster":"None"
                        }
    
    cluster = [i.text.strip() for i in tt.find_all('h4')]
    ci = -1
    asdf = pd.DataFrame(stds).T
    new_rows = pd.DataFrame(columns = asdf.columns)
    pgr = ""
    par = ""
    for i, row in asdf.iterrows():
      if pgr != row['grade'] or par != row['area']:
        ci +=1
        nr = copy.deepcopy(row)
        nr.name = nr.grade + "." + nr.domain + "." + nr.area
        nr.strand = ""
        nr.substrand = ""
        nr.examples = ""
        nr.full_text = ""
        nr.cluster = cluster[ci]
        nr.text = cluster[ci]
        new_rows = pd.concat([new_rows,pd.DataFrame(nr).T])

      row['cluster'] = cluster[ci]
      pgr = row['grade']
      par = row['area']
  
    ssdf = pd.concat([ssdf,asdf,new_rows]).drop_duplicates()
  if len(try_again) > 0:
    urls = try_again
    try_again = []
  else:
    do_scraping = False

ssdf.to_csv(DATA_FOLDER + "cc_text_all.csv")
ssdf

t/G/
/MD/
/NF/
/EE/
t/G/


,grade,domain,area,strand,substrand,cluster,text,examples,full_text
K.CC.A.1,K,CC,A,1,,Know number names and the count sequence.,Count to 100 by ones and by tens.,,CCSS.Math.Content.K.CC.A.1Count to 100 by ones...
K.CC.A.2,K,CC,A,2,,Know number names and the count sequence.,Count forward beginning from a given number wi...,,CCSS.Math.Content.K.CC.A.2Count forward beginn...
K.CC.A.3,K,CC,A,3,,Know number names and the count sequence.,Write numbers from 0 to 20. Represent a number...,,CCSS.Math.Content.K.CC.A.3Write numbers from 0...
K.CC.B.4,K,CC,B,4,,Count to tell the number of objects.,Understand the relationship between numbers an...,,CCSS.Math.Content.K.CC.B.4Understand the relat...
K.CC.B.5,K,CC,B,5,,Count to tell the number of objects.,"Count to answer ""how many?"" questions about as...",,"CCSS.Math.Content.K.CC.B.5Count to answer ""how..."
...,...,...,...,...,...,...,...,...,...
7.G.A,7,G,A,,,"Draw construct, and describe geometrical figur...","Draw construct, and describe geometrical figur...",,
7.G.B,7,G,B,,,Solve real-life and mathematical problems invo...,Solve real-life and mathematical problems invo...,,
8.G.A,8,G,A,,,Understand congruence and similarity using phy...,Understand congruence and similarity using phy...,,
8.G.B,8,G,B,,,Understand and apply the Pythagorean Theorem.,Understand and apply the Pythagorean Theorem.,,


In [ ]:
import copy

In [ ]:
pd.DataFrame(nr).T

,K.CC.C
grade,K
domain,CC
area,C
strand,
substrand,
text,Compare numbers.
examples,
full_text,
cluster,Compare numbers.


In [ ]:
cluster = [i.text.strip() for i in tt.find_all('h4')]
ci = -1
asdf = sdf[sdf.domain == 'SP']
asdf['cluster'] = "None"
pgr = ""
par = ""
for i, row in asdf.iterrows():
  if pgr != row['grade'] or par != row['area']:
    ci += 1
  row['cluster'] = cluster[ci]
  pgr = row['grade']
  par = row['area']
  


asdf
# sdf[sdf.domain == 'SP']

<ipython-input-152-6676546c4f29>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asdf['cluster'] = "None"


,grade,domain,area,strand,substrand,text,examples,full_text,cluster
6.SP.A.1,6,SP,A,1,,Recognize a statistical question as one that a...,"For example, ""How old am I?"" is not a statisti...",CCSS.Math.Content.6.SP.A.1Recognize a statisti...,Develop understanding of statistical variability.
6.SP.A.2,6,SP,A,2,,Understand that a set of data collected to ans...,,CCSS.Math.Content.6.SP.A.2Understand that a se...,Develop understanding of statistical variability.
6.SP.A.3,6,SP,A,3,,Recognize that a measure of center for a numer...,,CCSS.Math.Content.6.SP.A.3Recognize that a mea...,Develop understanding of statistical variability.
6.SP.B.4,6,SP,B,4,,Display numerical data in plots on a number li...,,CCSS.Math.Content.6.SP.B.4Display numerical da...,Summarize and describe distributions.
6.SP.B.5,6,SP,B,5,,Summarize numerical data sets in relation to t...,,CCSS.Math.Content.6.SP.B.5Summarize numerical ...,Summarize and describe distributions.
7.SP.A.1,7,SP,A,1,,Understand that statistics can be used to gain...,,CCSS.Math.Content.7.SP.A.1Understand that stat...,Use random sampling to draw inferences about a...
7.SP.A.2,7,SP,A,2,,Use data from a random sample to draw inferenc...,"For example, estimate the mean word length in ...",CCSS.Math.Content.7.SP.A.2Use data from a rand...,Use random sampling to draw inferences about a...
7.SP.B.3,7,SP,B,3,,Informally assess the degree of visual overlap...,"For example, the mean height of players on the...",CCSS.Math.Content.7.SP.B.3Informally assess th...,Draw informal comparative inferences about two...
7.SP.B.4,7,SP,B,4,,Use measures of center and measures of variabi...,"For example, decide whether the words in a cha...",CCSS.Math.Content.7.SP.B.4Use measures of cent...,Draw informal comparative inferences about two...
7.SP.C.5,7,SP,C,5,,Understand that the probability of a chance ev...,,CCSS.Math.Content.7.SP.C.5Understand that the ...,"Investigate chance processes and develop, use,..."


['Develop understanding of statistical variability.  ',
 'Summarize and describe distributions.  ',
 'Use random sampling to draw inferences about a population.  ',
 'Draw informal comparative inferences about two populations.  ',
 'Investigate chance processes and develop, use, and evaluate probability models.  ',
 'Investigate patterns of association in bivariate data.  ']

In [ ]:
sdf = pd.DataFrame(columns = ["grade",
                        "domain",
                        "area",
                        "strand",
                        "substrand",
                        "text",
                        "examples",
                        "full_text"])

,grade,domain,area,strand,substrand,text,examples,full_text
K.OA.A.1,K,OA,A,1,,Represent addition and subtraction with object...,,CCSS.Math.Content.K.OA.A.1Represent addition a...
K.OA.A.2,K,OA,A,2,,"Solve addition and subtraction word problems, ...",,CCSS.Math.Content.K.OA.A.2Solve addition and s...
K.OA.A.3,K,OA,A,3,,Decompose numbers less than or equal to 10 int...,,CCSS.Math.Content.K.OA.A.3Decompose numbers le...
K.OA.A.4,K,OA,A,4,,"For any number from 1 to 9, find the number th...",,CCSS.Math.Content.K.OA.A.4For any number from ...
K.OA.A.5,K,OA,A,5,,Fluently add and subtract within 5,,CCSS.Math.Content.K.OA.A.5Fluently add and sub...
1.OA.A.1,1,OA,A,1,,Use addition and subtraction within 20 to solv...,,CCSS.Math.Content.1.OA.A.1Use addition and sub...
1.OA.A.2,1,OA,A,2,,Solve word problems that call for addition of ...,,CCSS.Math.Content.1.OA.A.2Solve word problems ...
1.OA.B.3,1,OA,B,3,,Apply properties of operations as strategies t...,"Examples: If 8 + 3 = 11 is known, then 3 + 8 =...",CCSS.Math.Content.1.OA.B.3Apply properties of ...
1.OA.B.4,1,OA,B,4,,Understand subtraction as an unknown-addend pr...,"For example, subtract 10 - 8 by finding the nu...",CCSS.Math.Content.1.OA.B.4Understand subtracti...
1.OA.C.5,1,OA,C,5,,Relate counting to addition and subtraction (e...,,CCSS.Math.Content.1.OA.C.5Relate counting to a...


In [ ]:
print(page_data.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>
<html lang="en-US" class="no-js lt-ie9 lt-ie8 lt-ie7"><![endif]-->
<!--[if (IE 7)&!(IEMobile)]>
<html lang="en-US" class="no-js lt-ie9 lt-ie8"><![endif]-->
<!--[if (IE 8)&!(IEMobile)]>
<html lang="en-US" class="no-js lt-ie9"><![endif]-->
<!--[if gt IE 8]><!-->
<html lang="en-US">
 <!--<![endif]-->
 <head>
  <script charset="utf-8" src="/_static/js/bundle-playback.js?v=21L7o4JU" type="text/javascript">
  </script>
  <script charset="utf-8" src="/_static/js/wombat.js?v=Jjml7g96" type="text/javascript">
  </script>
  <script type="text/javascript">
   __wm.init("https://web.archive.org/web");
  __wm.wombat("http://www.corestandards.org/Math/Content/OA/","20220206180624","https://web.archive.org/","web","/_static/",
	      "1644170784");
  </script>
  <link href="/_static/css/banner-styles.css?v=S1zqJCYt" rel="stylesheet" type="text/css"/>
  <link href="/_static/css/iconochive.css?v=qtvMKcIJ" rel="stylesheet" type="text/css"/>
  <!-- End Wayback Rewrite JS

In [ ]:
page_data

<!DOCTYPE html>
<!--[if lt IE 7]>
<html lang="en-US" class="no-js lt-ie9 lt-ie8 lt-ie7"><![endif]--><!--[if (IE 7)&!(IEMobile)]>
<html lang="en-US" class="no-js lt-ie9 lt-ie8"><![endif]--><!--[if (IE 8)&!(IEMobile)]>
<html lang="en-US" class="no-js lt-ie9"><![endif]--><!--[if gt IE 8]><!--><html lang="en-US"><!--<![endif]--><head><script charset="utf-8" src="/_static/js/bundle-playback.js?v=21L7o4JU" type="text/javascript"></script>
<script charset="utf-8" src="/_static/js/wombat.js?v=Jjml7g96" type="text/javascript"></script>
<script type="text/javascript">
  __wm.init("https://web.archive.org/web");
  __wm.wombat("http://www.corestandards.org/Math/Content/OA/","20220206180624","https://web.archive.org/","web","/_static/",
	      "1644170784");
</script>
<link href="/_static/css/banner-styles.css?v=S1zqJCYt" rel="stylesheet" type="text/css"/>
<link href="/_static/css/iconochive.css?v=qtvMKcIJ" rel="stylesheet" type="text/css"/>
<!-- End Wayback Rewrite JS Include -->

		<meta charset=

In [ ]:
sdf['domain_code'] = sdf.standard_code.apply(lambda x: x.split(".")[1])
sdf['gr'] = sdf.standard_code.apply(lambda x: x.split(".")[0])
sdf['std_num'] =  sdf.standard_code.apply(lambda x: x.split(".")[2].split(" ")[0])
sdf['sub_std'] =  sdf.standard_code.apply(lambda x: x.split(".")[2].split(" ")[-1].upper())

In [ ]:
sdf.iloc[22].standard_code = "1.NBT.4"


In [ ]:
sdf.other.apply(lambda x:x.split(" "))

0     [1]
1     [2]
2     [3]
3     [4]
4     [5]
     ... 
28    [2]
29    [3]
30    [4]
31    [5]
32    [6]
Name: other, Length: 389, dtype: object

In [ ]:
sdf.to_csv(DATA_FOLDER + "ccss_text.csv")

In [ ]:
sdf.cluster_statement.value_counts()

The Standards for Mathematical Practice describe varieties of expertise that mathematics educators at all levels should seek to develop in their students.    72
Apply and extend previous understandings of multiplication and division to multiply and divide fractions.                                                     12
Understand place value.                                                                                                                                       11
Apply and extend previous understandings of operations with fractions to add, subtract, multiply, and divide rational numbers.                                11
Apply and extend previous understandings of numbers to the system of rational numbers.                                                                        11
                                                                                                                                                              ..
Generate and analyze patterns.    

In [ ]:
p = page_data
# for a in p.find_next('div'):
#   print(a.text)
# p.find_next('div')
[x.text.strip() for x in p.find_all('h2')]
[x.text.strip() for x in p.find_all('h4')]
[x.text.strip() for x in p.find_all('div','standard')]


['CCSS.Math.Content.K.OA.A.1Represent addition and subtraction with objects, fingers, mental images, drawings1, sounds (e.g., claps), acting out situations, verbal explanations, expressions, or equations.',
 'CCSS.Math.Content.K.OA.A.2Solve addition and subtraction word problems, and add and subtract within 10, e.g., by using objects or drawings to represent the problem.',
 'CCSS.Math.Content.K.OA.A.3Decompose numbers less than or equal to 10 into pairs in more than one way, e.g., by using objects or drawings, and record each decomposition by a drawing or equation (e.g., 5 = 2 + 3 and 5 = 4 + 1).',
 'CCSS.Math.Content.K.OA.A.4For any number from 1 to 9, find the number that makes 10 when added to the given number, e.g., by using objects or drawings, and record the answer with a drawing or equation.',
 'CCSS.Math.Content.K.OA.A.5Fluently add and subtract within 5.',
 'CCSS.Math.Content.1.OA.A.1Use addition and subtraction within 20 to solve word problems involving situations of adding t

In [ ]:
page_data

<!DOCTYPE html>
<!--[if lt IE 7]>
<html lang="en-US" class="no-js lt-ie9 lt-ie8 lt-ie7"><![endif]--><!--[if (IE 7)&!(IEMobile)]>
<html lang="en-US" class="no-js lt-ie9 lt-ie8"><![endif]--><!--[if (IE 8)&!(IEMobile)]>
<html lang="en-US" class="no-js lt-ie9"><![endif]--><!--[if gt IE 8]><!--><html lang="en-US"><!--<![endif]--><head><script charset="utf-8" src="/_static/js/bundle-playback.js?v=21L7o4JU" type="text/javascript"></script>
<script charset="utf-8" src="/_static/js/wombat.js?v=Jjml7g96" type="text/javascript"></script>
<script type="text/javascript">
  __wm.init("https://web.archive.org/web");
  __wm.wombat("http://www.corestandards.org/Math/Content/OA/","20220206180624","https://web.archive.org/","web","/_static/",
	      "1644170784");
</script>
<link href="/_static/css/banner-styles.css?v=S1zqJCYt" rel="stylesheet" type="text/css"/>
<link href="/_static/css/iconochive.css?v=qtvMKcIJ" rel="stylesheet" type="text/css"/>
<!-- End Wayback Rewrite JS Include -->

		<meta charset=

# Another free resource for high quality teacher language

In [ ]:
url= "https://flexbooks.ck12.org/cbook/ck-12-fifth-grade-math-resource-flexlet/"
url = "https://flexbooks.ck12.org/cbook/ck-12-fourth-grade-math-resource-flexlet/"

## 5th and 4th grades

## check this one for math and image text rendering in bs4
url = "https://flexbooks.ck12.org/cbook/ck-12-middle-school-math-concepts-grade-6/section/3.3/related/lesson/decimals-in-words/"

## 6th grade (first is on grade, the second is basic, the third is interactive, and the fourth is preparation for MS)
url = "https://flexbooks.ck12.org/cbook/ck-12-middle-school-math-concepts-grade-6/"
url = "https://www.ck12.org/book/ck-12-middle-school-math-grade-6/"
url = "https://flexbooks.ck12.org/cbook/ck-12-cbse-maths-class-6/"
url = "https://flexbooks.ck12.org/cbook/prep-for-7th-grade-math/"

## These two have a mix of grades
url = "https://www.ck12.org/book/algebra-explorations-concepts-pre-k-through-grade-7/"
url = "https://www.ck12.org/book/algebra-explorations-pre-k-through-grade-7/"

In [ ]:
url = "https://flexbooks.ck12.org/cbook/ck-12-middle-school-math-concepts-grade-6/section/3.3/related/lesson/decimals-in-words/"
r1 = requests.get(url)
page = r1.content
page_data = BeautifulSoup(page, 'html5lib')

# t = page_data.find('main')
print(page_data.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   CK12-Foundation
  </title>
  <link href="//static1.ck12.org/media/build-20230125015016/course-book-app/dist/main.css" rel="stylesheet"/>
  <link as="font" crossorigin="anonymous" href="//static2.ck12.org/media/build-20230125015016/common/font/proximanova/bold.woff" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="anonymous" href="//static2.ck12.org/media/build-20230125015016/common/font/proximanova/semi_bold.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="anonymous" href="//static2.ck12.org/media/build-20230125015016/common/font/proximanova/regular.woff" rel="preload" type="font/woff2"/>
  <link href="/favicon.svg" rel="shortcut icon" type="image/x-icon"/>
  <!-- Common meta tags -->
  <meta content="CK-12 Foundation" itemprop="name" name="author" property="og:site_name"/>
  <meta content="159884644067996" property="fb:app_id"/>
  <meta content="https://www.ck12.org" itemprop="url"/>
  <meta content="(6

In [ ]:
asdf = [x.text for x in page_data.find_all('a')]
asdf

['Video',
 'Map',
 'Search',
 'Common Core State Standards Initiative',
 'Common Core State Standards Initiative',
 'Home',
 'About the Standards',
 'Development Process',
 'Frequently Asked Questions',
 'Myths vs. Facts',
 'Branding Guidelines',
 'Contact',
 'What Parents Should Know',
 'Standards in Your State',
 'Read the Standards',
 'English Language Arts Standards',
 'Mathematics Standards',
 'Other Resources',
 'Key Shifts in English Language Arts',
 'Key Shifts in Mathematics',
 'Statements of Support',
 'Print this page',
 'CCSS.Math.Content.K.OA.A.1',
 'CCSS.Math.Content.K.OA.A.2',
 'CCSS.Math.Content.K.OA.A.3',
 'CCSS.Math.Content.K.OA.A.4',
 'CCSS.Math.Content.K.OA.A.5',
 'CCSS.Math.Content.1.OA.A.1',
 'CCSS.Math.Content.1.OA.A.2',
 'CCSS.Math.Content.1.OA.B.3',
 'CCSS.Math.Content.1.OA.B.4',
 'CCSS.Math.Content.1.OA.C.5',
 'CCSS.Math.Content.1.OA.C.6',
 'CCSS.Math.Content.1.OA.D.7',
 'CCSS.Math.Content.1.OA.D.8',
 'CCSS.Math.Content.2.OA.A.1',
 'CCSS.Math.Content.2.OA.B.2'

# Code for Getting the EngageNY stuff!

In [ ]:




# ----auto-download for engageNY curriculum---
def engage_NY(url, folder_location, keywords):

    if not os.path.exists(folder_location):os.mkdir(folder_location)

    for i, k in enumerate(keywords):
        if i>=0:
            link1=urljoin(url,k)
            print(link1)
            response = requests.get(link1)
            soup= BeautifulSoup(response.text, "html5lib")  
            for i, link in enumerate(soup.select("a[href]")):
                link_list=[]
                if 'module' in link['href']:
                    link_list.append(link['href'])
                for l in link_list:
                    join2=urljoin(url,l)
                    res2=requests.get(join2)
                    soup2= BeautifulSoup(res2.text, "html5lib")
                    for i, link in enumerate(soup2.select("a[href]")):
                        if '.pdf' in link['href']:
                            join3=urljoin(url,link['href'])
                            print(join3)
                            marker=link['href'].split('?')[0].split('/')[-1]
                            filename = os.path.join(folder_location,marker)
                            print(filename)
                            with open(filename, 'wb') as f:
                                f.write(requests.get(join3).content)
                                f.close()


# ----convert pdf to text-----
def pdfparser(in_path,out_path):

    fp = open(in_path, 'rb')
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec ='utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr,  laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.

    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()
        with open(out_path,'w',encoding=codec) as f:
            f.write(data)
            
            f.close()
    return data

# to execute                         
url = "https://www.engageny.org/"
grade_level = 5
url = f"https://nysed.sharepoint.com/sites/P12EngageNY-Math-EXTA/Shared%20Documents/Forms/AllItems.aspx?ga=1&id=%2Fsites%2FP12EngageNY%2DMath%2DEXTA%2FShared%20Documents%2FMathematics%2FGrade%20{grade_level}&viewid=74a2b97e%2D3088%2D44b7%2Dab73%2D420870c488b4"
folder_location = DATA_FOLDER + 'engageNY/G5/'
keywords=['resource/grade-1-mathematics','resource/grade-2-mathematics','resource/grade-3-mathematics',
          'resource/grade-4-mathematics','resource/grade-5-mathematics','resource/grade-6-mathematics',
          'resource/grade-7-mathematics','resource/kindergarten-mathematics','content/prekindergarten-mathematics',
          'resource/high-school-algebra-i','resource/high-school-algebra-ii','resource/high-school-geometry',
          'content/precalculus-and-advanced-topics','resource/grade-8-mathematics']
keywords=['resource/grade-3-mathematics',
          'resource/grade-4-mathematics','resource/grade-5-mathematics','resource/grade-6-mathematics',]
keywords=['']
engage_NY(url, folder_location, keywords)

# possibly do each separately




https://nysed.sharepoint.com/sites/P12EngageNY-Math-EXTA/Shared%20Documents/Forms/AllItems.aspx?ga=1&id=%2Fsites%2FP12EngageNY%2DMath%2DEXTA%2FShared%20Documents%2FMathematics%2FGrade%205&viewid=74a2b97e%2D3088%2D44b7%2Dab73%2D420870c488b4


In [ ]:
url5 = "https://nysed.sharepoint.com/sites/P12EngageNY-Math-EXTA/Shared%20Documents/Forms/AllItems.aspx?ga=1&id=%2Fsites%2FP12EngageNY%2DMath%2DEXTA%2FShared%20Documents%2FMathematics%2FGrade%205&viewid=74a2b97e%2D3088%2D44b7%2Dab73%2D420870c488b4"

In [ ]:
url4 = "https://nysed.sharepoint.com/sites/P12EngageNY-Math-EXTA/Shared%20Documents/Forms/AllItems.aspx?ga=1&id=%2Fsites%2FP12EngageNY%2DMath%2DEXTA%2FShared%20Documents%2FMathematics%2FGrade%204&viewid=74a2b97e%2D3088%2D44b7%2Dab73%2D420870c488b4"

In [ ]:
print(np.version.version)


1.21.6


In [ ]:
if not os.path.exists(folder_location):os.mkdir(folder_location)

In [ ]:
response

NameError: ignored

In [ ]:
# print(url)
response = requests.get(url)
soup= BeautifulSoup(response.text, "html5lib")  

In [ ]:
soup 

<html><head></head><body>403 FORBIDDEN</body></html>